In [1]:
import natasha
from natasha import Segmenter, NewsEmbedding, NewsMorphTagger, NewsSyntaxParser, Doc, NewsNERTagger
from natasha import PER, NamesExtractor, MorphVocab
from natasha import NamesExtractor

from navec import Navec
from slovnet import NER
from ipymarkup import show_span_ascii_markup as show_span

import pandas as pd

from yargy import Parser, rule, and_, or_
from yargy.predicates import gram, dictionary, eq, length_eq
from yargy.pipelines import morph_pipeline

In [2]:
def check_reply(text, wchtx):
    
    if wchtx == 'greeting':
        RULE = morph_pipeline(['добрый день', 'добрый вечер', 'здравствуйте', 'приветствую'])
    
    elif wchtx == 'byby':
        RULE = morph_pipeline(['до свидания', 'до встречи', 'всего доброго', 'всего хорошего', 'всего вам хорошего', 'до связи', 'на связи'])
    
    elif wchtx == 'org_name':
        RULE = or_(rule(dictionary({'компания', 'организация', 'фирма'}), (gram('Name'))),
                   rule(dictionary({'компания', 'организация', 'фирма'}), (gram('NOUN')))
                  )
    elif wchtx == 'person_name':
        #RULE = rule(gram('Name'), and_(gram('NOUN')))
        #RULE =rule(dictionary({'зовут','имя', 'это'}), and_(gram('Name')))
        RULE =rule(dictionary({'зовут','имя', 'это', 'меня'}), and_(gram('Name')))
    
    parser = Parser(RULE)
    
    if len(list((parser.findall(text)))):
        return True
    else:
        return False
    

In [3]:
def get_replay(text, wchtx):
    items_reply = []

    if wchtx == 'org_name':
        RULE = or_(rule(dictionary({'компания', 'организация', 'фирма'}), (gram('Name'))),
                   rule(dictionary({'компания', 'организация', 'фирма'}), (gram('NOUN')))
                  )

    elif wchtx == 'person_name':
        #RULE = rule(gram('Name'), and_(gram('NOUN')))
        RULE =rule(dictionary({'зовут','имя', 'это', 'меня'}), and_(gram('Name')))
        
    parser = Parser(RULE)
    
    for match in parser.findall(text):
        items_reply = ([x.value for x in match.tokens])
    
    return items_reply

In [4]:
df = pd.read_csv('./test_data.csv')
df.head(5)

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага


In [5]:
df['greeting'] =''
df['farewell'] = ''
df['company_name'] = ''
df['manager_name'] = ''
df['check_manager_rule'] = ''
df['manager_name'] = ''

In [6]:
df.head(10)

,dlg_id,line_n,role,text,greeting,farewell,company_name,manager_name,check_manager_rule
0,0,0,client,Алло,,,,,
1,0,1,manager,Алло здравствуйте,,,,,
2,0,2,client,Добрый день,,,,,
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,,,,,
4,0,4,client,Ага,,,,,
5,0,5,manager,Угу ну возможно вы рассмотрите и другие вариан...,,,,,
6,0,6,client,Да мы работаем с компанией которая нам подлива...,,,,,
7,0,7,client,Как как бы уже до этого момента работаем все у...,,,,,
8,0,8,manager,Угу а на что вы обращаете внимание при выборе,,,,,
9,0,9,client,Как бы нет,,,,,


In [7]:
dialogs = list(set(list(df['dlg_id'].values)))
dialogs

[0, 1, 2, 3, 4, 5]

In [8]:
for dialog in  dialogs:
     
    # проверка для Менеджера, поздороваться и попрощался
    check_manager_rules = 0
    
    for row in df[(df['dlg_id'] == dialog) & (df['role'] == 'manager')].itertuples():
        #df.loc[row.Index, 'greeting'] = 'testing'
        #print(row.Index, row.role, row.text)
    
        if check_reply(row.text, 'greeting'):
            check_manager_rules += 1
            #print(f"Реплика: {row.text}, тип - greeting")
            df.loc[row.Index, 'greeting'] = row.text

        if check_reply(row.text, 'byby'):
            check_manager_rules += 1
            #print(f"Реплика: {row.text}, тип - byby")
            df.loc[row.Index, 'farewell'] = row.text

        if check_reply(row.text, 'org_name'):
            #print(f"Реплика: {row.text}, тип - org_name")
            #print(f"Компания: {get_replay(row.text, 'org_name')}")
            company_name = ' '.join(get_replay(row.text, 'org_name'))
            df.loc[row.Index, 'company_name'] = company_name

        if check_reply(row.text, 'person_name'):
            #print(f"Реплика: {row.text}, тип - person_name")
            #print(f"Имя менеджера: {get_replay(row.text, 'person_name')}")
            person_name = ' '.join(get_replay(row.text, 'person_name'))
            df.loc[row.Index, 'manager_name'] = person_name
            
    if check_manager_rules >= 2:
        df.loc[row.Index, 'check_manager_rule'] = 1
        print(f"Менеджер выполнил правило.")
    print(f"Диалог № {dialog} парсинг закончен.")

Менеджер выполнил правило.
Диалог № 0 парсинг закончен.
Менеджер выполнил правило.
Диалог № 1 парсинг закончен.
Менеджер выполнил правило.
Диалог № 2 парсинг закончен.
Менеджер выполнил правило.
Диалог № 3 парсинг закончен.
Диалог № 4 парсинг закончен.
Менеджер выполнил правило.
Диалог № 5 парсинг закончен.
